In [27]:
from zipfile import ZipFile
import os

# Define the path to the zip file and the extraction directory
zip_file_path = '/Final Data.zip'
extraction_dir = '/Data'
# extraction_dir = '/Final Data'

# Create a directory for the extracted files
os.makedirs(extraction_dir, exist_ok=True)

# Extract the zip file
with ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extraction_dir)

# List the files in the extraction directory
extracted_files = os.listdir(extraction_dir)
extracted_files


['Final Data', '__MACOSX']

In [28]:
# Define the path to the 'Final Data' folder
final_data_dir = os.path.join(extraction_dir, 'Final Data')
# final_data_dir = os.path.join(extraction_dir)

# List the files and directories in the 'Final Data' folder
final_data_contents = os.listdir(final_data_dir)
final_data_contents


['bee', 'antelope', 'badger', 'bat', 'bear', '.DS_Store']

In [29]:
# Define the path to one of the class directories
sample_class_dir = os.path.join(final_data_dir, 'bee')

# List the first few files in the directory
sample_files = os.listdir(sample_class_dir)
sample_files[:5]


['517aca5da2.jpg',
 '160cdad8e6.jpg',
 '2afadcf34f.jpg',
 '5adc0a316b.jpg',
 '81c541cae0.jpg']

In [7]:
pip install torch torchvision


In [19]:
import torch
from torchvision import transforms
from torchvision.io import read_image
from torch.utils.data import Dataset, DataLoader, random_split
import os
import glob

class OneVsRestDataset(Dataset):
    def __init__(self, data_dir, target_class, transform=None):
        """
        Args:
            data_dir (string): Directory with all the images.
            target_class (string): The class to be considered as '1' (positive class).
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.data_dir = data_dir
        self.target_class = target_class
        self.transform = transform
        self.samples = []
        print(f"Looking in directory: {data_dir}")  # Debug print
        if not os.path.exists(data_dir):
            print(f"Directory does not exist: {data_dir}")
            return

        # Load all image file paths and their labels
        for class_name in os.listdir(data_dir):
            class_dir = os.path.join(data_dir, class_name)
            if not os.path.isdir(class_dir):
                continue  # Skip non-directory files
            print(f"Processing class directory: {class_name}")
            for img_path in glob.glob(os.path.join(class_dir, '*.jpg')):
                # Label is 1 for the target class, and 0 for all other classes
                label = 1 if class_name == target_class else 0
                self.samples.append((img_path, label))

        print(f"Total samples loaded: {len(self.samples)}")

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        image = read_image(img_path)  # Load image
        image = image.float() / 255.0  # Normalize to [0, 1]

        if self.transform:
            image = self.transform(image)

        return image, label


In [13]:
print(f"Total samples loaded: {len(dataset)}")

Total samples loaded: 0


In [30]:
# Define your transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to 224x224 for models like ResNet
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # ImageNet norms
])

# Initialize the dataset
dataset = OneVsRestDataset(data_dir='/Data/Final Data', target_class='bee', transform=transform)

# Splitting the dataset for 3-fold cross-validation
# fold_size = len(dataset) // 3
# datasets = random_split(dataset, [fold_size, fold_size, len(dataset) - 2 * fold_size])
if len(dataset) > 0:
    fold_size = len(dataset) // 3
    datasets = random_split(dataset, [fold_size, fold_size, len(dataset) - 2 * fold_size])
    # You can now proceed with DataLoader initialization
else:
    print("No samples loaded. Please check the dataset path and contents.")


# Example: DataLoader for the first fold
# train_loader = DataLoader(datasets[0], batch_size=32, shuffle=True)
if len(datasets[0]) > 0:
    train_loader = DataLoader(datasets[0], batch_size=32, shuffle=True)
    # Do similarly for the other folds
else:
    print("The first fold is empty.")



Looking in directory: /Data/Final Data
Processing class directory: bee
Processing class directory: antelope
Processing class directory: badger
Processing class directory: bat
Processing class directory: bear
Total samples loaded: 300
